In [49]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Dense
from keras.preprocessing import image

image_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                              height_shift_range=0.1, rescale=1/255,
                              shear_range=0.2, zoom_range=0.2,
                              horizontal_flip=True, fill_mode='nearest')



input_shape = (150,150,3)
 #create the model
model = Sequential()

#add in layers:1
model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#add in layers:2
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#add in layers:3
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu')) #you can swap out different activations here

#add dropout to reduce overfitting
model.add(Dropout(0.5))

#add last output. Here it's just one neuron because this is binary target class (0->cat or 1->dog)
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=['accuracy'])

model.summary()





Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 18496)           

In [30]:
#train the model
batch_size = 16
#generate many manipulated images directly from the directory
train_image_gen = image_gen.flow_from_directory('../DATA/CATS_DOGS/train',
                                                target_size=input_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')


#generate many manipulated images directly from the directory
test_image_gen = image_gen.flow_from_directory('../DATA/CATS_DOGS/test',
                                                target_size=input_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [31]:
train_image_gen.class_indices

{'cats': 0, 'dogs': 1}

In [35]:
result = model.fit_generator(train_image_gen,epochs=1, steps_per_epoch=150,
                             validation_data=test_image_gen,validation_steps=12)

Epoch 1/1
150/150 [==============================] - 62s 413ms/step - loss: 0.6974 - accuracy: 0.5058 - val_loss: 0.6953 - val_accuracy: 0.6354


In [38]:
#evaluate the model
result.history['val_accuracy']

[0.6354166865348816]

In [53]:
#predict on new images
dog_file = '../DATA/CATS_DOGS/test/dogs/dog.1506.jpg'

#resize the image that the neural network expects it in
dog_img = image.load_img(dog_file, target_size=(150,150))

#turn image into and array
dog_img = image.img_to_array(dog_img)

#dog_img.shape = (150, 150, 3) :change this so that the neural netwok thinks its a batch on 1 image
#so we need the dimensions to be 1,150,150,3

dog_img = np.expand_dims(dog_img,axis=0)

#make sure all the values are between 0 and 1
dog_img = dog_img/255

model.predict_classes(dog_img)


array([[1]])

In [54]:
#check how sure it was
model.predict(dog_img)

array([[0.5342044]], dtype=float32)